###  Graph stock prices using using candle sticks

In [ ]:
import datetime as dt
import matplotlib.pyplot as plt
from matplotlib import style
import pandas as pd
import pandas_datareader.data as web
from datetime import date

import altair as alt
from vega_datasets import data

from mpl_finance import candlestick_ohlc
import matplotlib.dates as mdates
import random
from datetime import timedelta

In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:80% !important; }</style>"))

In [ ]:
def stock_df(ticker,source,start,end):
    
    df = web.DataReader(ticker,source,start,end)
    df['%change'] = (df['Close']-df['Open'])/df['Open']*100
    
    return df

def altair_candle(start,end,ticker,moving_av):
    
    if len(moving_av)!=0:
        start = start -timedelta(days=max(moving_av))
    df=stock_df(ticker,'yahoo',start,end)
    df.reset_index(inplace=True)
    
    interval = alt.selection_interval(encodings=['x'])
    open_close_color = alt.condition("datum.Open <= datum.Close",
                                 alt.value("#06982d"),
                                 alt.value("#980617"))
    
    base = alt.Chart(df).encode(
        alt.X('Date:T',
              axis=alt.Axis(
                  format='%m/%d/%y',labelAngle=-45),
                  title=ticker +' stocks from '+str(start)[0:10]+' to '+str(end)[0:10]),
        color=open_close_color, 
        tooltip = [alt.Tooltip('Date:T'), alt.Tooltip('High:Q',format='.2f'),alt.Tooltip('Low:Q',format='.2f'),
                   alt.Tooltip('Open:Q',format='.2f'),alt.Tooltip('Close:Q',format='.2f'),
                  alt.Tooltip('%change:Q',format='.2f')] 
        ).properties(
        width=800,
        height=350,
        title=ticker)

    rule = base.mark_rule().encode(
        alt.Y(
            'Low:Q',
            title='Price',
            scale=alt.Scale(zero=False),),
        alt.Y2('High:Q'))

    bar = base.mark_bar().encode(
        alt.Y('Open:Q'),
        alt.Y2('Close:Q')
    ).properties(
        selection=interval
    ).interactive()
    
    top = rule + bar
    
    ## adding any moving avergae
    if len(moving_av)!=0:
        for day_ma in moving_av:
            df[str(day_ma)+'ma'] = df['Close'].rolling(window=day_ma).mean()
            df.dropna(inplace=True)
            r = lambda: random.randint(0,255)
            para = str(day_ma)+ 'ma:Q'
            ma = base.mark_line(
            ).encode(
                alt.Y(para),
                color=alt.value('#%02X%02X%02X' % (r(),r(),r())) 
            ) 
            top = top + ma   
    
    ##add trade volume bar chart
    volume = add_trade_volume(df,interval,ticker,open_close_color)
    
    ## add percentage summary chart
    percentage = add_percentage_change(df,interval,ticker,open_close_color)
    
    return top&percentage&volume

def add_trade_volume(df,interval,ticker,open_close_color):
    bottom = alt.Chart(df).mark_bar().encode(
            alt.X('Date:T',title = ticker+' Trading Volume'),
            alt.Y('Volume:Q'),
            color=open_close_color,
            tooltip = [alt.Tooltip('Date:T'), alt.Tooltip('Volume:Q',format='.0f')]
        ).properties(
            width=800,
            height=100).transform_filter(interval).interactive()
    return bottom    

def add_percentage_change(df,interval,ticker,open_close_color):
    percentage = alt.Chart(df).mark_bar().encode(
            alt.X('Date:T',title = ticker+' Trading Volume'),
            alt.Y('%change:Q'),
            color=open_close_color,
            tooltip = [alt.Tooltip('Date:T'), alt.Tooltip('%change:Q',format='.2f')]
        ).properties(
            width=800,
            height=200).transform_filter(interval).interactive()
    return percentage

In [ ]:
start = dt.datetime(2020,1,1)
end = date.today()
altair_candle(start,end,'NIO',[20,50])

In [ ]:
## return a S&P500 list

import bs4 as bs
import pickle
import requests

def save_sp500_tickers():
    resp = requests.get('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')
    soup = bs.BeautifulSoup(resp.text,'lxml')
    table = soup.find('table', {'id': 'constituents'})
    tickers=[]
    
    for row in table.findAll('tr')[1:]:
        ticker = row.find('td').text
        tickers.append(ticker)
        
    with open("sp500tickers.pickle","wb") as f:
        pickle.dump(tickers,f)
    
    ticks500 = []
    for ticks in tickers:
        ticks500.append(ticks[slice(0,-1)])
        
    return ticks500

tickers = save_sp500_tickers()